In [ ]:
from printind.printind_function import printi, printiv
from Env_instance import resume_env, nb_actuations

import numpy as np
from tensorforce.agents import PPOAgent
from tensorforce.execution import Runner
import os
import csv

# Initialize and Define Environment

In [ ]:
printi("resume env")

environment = resume_env(plot=False, step=100, dump=100, first_run=False)
deterministic=True

printi("define network specs")

network_spec = [
    dict(type='dense', size=512),
    dict(type='dense', size=512),
]

printi("define agent")

printiv(environment.states)
printiv(environment.actions)
printiv(network_spec)

In [ ]:
printi("resume env")

environment = resume_env(plot=2000, step=2000, dump=100)
deterministic=True

printi("define network specs")

network_spec = [
    dict(type='dense', size=512),
    dict(type='dense', size=512),
]

printi("define agent")

printiv(environment.states)
printiv(environment.actions)
printiv(network_spec)

# Define PPO Agent

In [ ]:
agent = PPOAgent(
    states=environment.states(),
    actions=environment.actions(),
    network=network_spec,
    # Agent
    states_preprocessing=None,
    actions_exploration=None,
    reward_preprocessing=None,
    # MemoryModel
    update_mode=dict(
        unit='episodes',
        # 10 episodes per update
        batch_size=20,
        # Every 10 episodes
        frequency=20
    ),
    memory=dict(
        type='latest',
        include_next_states=False,
        capacity=10000
    ),
    # DistributionModel
    distributions=None,
    entropy_regularization=0.01,
    # PGModel
    baseline_mode='states',
    baseline=dict(
        type='mlp',
        sizes=[32, 32]
    ),
    baseline_optimizer=dict(
        type='multi_step',
        optimizer=dict(
            type='adam',
            learning_rate=1e-3
        ),
        num_steps=5
    ),
    gae_lambda=0.97,
    # PGLRModel
    likelihood_ratio_clipping=0.2,
    # PPOAgent
    step_optimizer=dict(
        type='adam',
        learning_rate=1e-3
    ),
    subsampling_fraction=0.2,
    optimization_steps=25,
    execution=dict(
        type='single',
        session_config=None,
        distributed_spec=None
    )
)

In [ ]:
def episode_finished(r):
    print("Finished episode {ep} after {ts} timesteps (reward: {reward})".format(ep=r.episode, ts=r.episode_timestep,
                                                                                 reward=r.episode_rewards[-1]))

    printi("save the mode")

    name_save = "./saved_models/ppo_model"
    r.agent.save_model(name_save, append_timestep=False)

    return True

In [ ]:
runner = Runner(agent=agent, environment=environment)

# Start Learning

In [ ]:
runner.run(episodes=250, max_episode_timesteps=nb_actuations, episode_finished=episode_finished)